#### This is a Look aside cache implementation for Aerospike and Mongo.

The basic cache is a read cache that reads value from the cache ( Aerospike )
If it doesn't find the value then it fetches the same value from Mongo and
inserts that record in Aerospike.

Further reads will then happen from Aerospike

Import all the dependencies from POM

In [1]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
  <dependency>
    <groupId>org.mongodb</groupId>
    <artifactId>mongo-java-driver</artifactId>
    <version>3.12.7</version>
  </dependency>
</dependencies>

In [2]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.mongodb.client.MongoDatabase;
import com.mongodb.client.MongoCollection;
import com.mongodb.MongoClient; 
import com.mongodb.MongoCredential; 
import org.bson.Document;
import com.mongodb.client.model.Filters;
import java.util.Set;

Since we are using an external instance of Mongo, It's best to launch that Database as a separate container.
To find the IP Address of that Container
`docker inspect <containerid> | grep -i ipaddress`

In [30]:
public class Cache{
        //Database Constants
        public static final String AEROSPIKE_HOST = "0.0.0.0";
        public static final String MONGO_HOST = "172.17.0.3";
        public static final int AEROSPIKE_PORT = 3000;
        public static final int MONGO_PORT = 27017;
        
        public static final String AEROSPIKE_NAMESPACE = "test";
        public static final String AEROSPIKE_SET = "demo";
        public static final String MONGO_USER = "sampleUser";
        public static final String MONGO_PASSWORD = "password";
        public static final String MONGO_DB = "myDb";
        public static final String MONGO_COLLECTION = "sampleCollection";
        
        private AerospikeClient client;
        private MongoClient mongo;
        private MongoCredential credential;
        private MongoDatabase database;
        
        public Cache() {
            client = new AerospikeClient(AEROSPIKE_HOST, AEROSPIKE_PORT);
            mongo = new MongoClient(MONGO_HOST , MONGO_PORT);
            credential = MongoCredential.createCredential(MONGO_USER, MONGO_DB, 
                                 MONGO_PASSWORD.toCharArray());
            database = mongo.getDatabase(MONGO_DB);
        }
        
        private boolean collectionExists(final String collectionName) {
           return database.listCollectionNames()
                .into(new ArrayList<String>()).contains(collectionName);
        }

        public void populateMongoData(String id, String data) {
            // Populate Mongodb first
            Document document = new Document(id, data);
            if (! collectionExists(MONGO_COLLECTION)) {
                database.createCollection(MONGO_COLLECTION);
            } else {
                MongoCollection<Document> collection = database.getCollection(MONGO_COLLECTION);
                collection.insertOne(document);
            }
            Key key = new Key(AEROSPIKE_NAMESPACE, AEROSPIKE_SET, id);
            client.delete(null, key);
        }
        
        public String getData(String id, String data) {
            /**
              This is just an example code that exhibits a cache fetch for a String id with
              String data
            */
            
            Key key = new Key(AEROSPIKE_NAMESPACE, AEROSPIKE_SET, id);
            String BIN_NAME = "value";
            Record record = client.get(null,key);
            if ( record == null ) {
                System.out.println("First Fetch Record does not exist in Aerospike cache");
                MongoCollection<Document> collection = database.getCollection(MONGO_COLLECTION);
                Document document = collection.find(Filters.eq(id, data)).first();
                //System.out.println("Document " + document.get(id));
                String json = document.get(id).toString();
                client.put(null, key, new Bin(BIN_NAME,json));
                return client.get(null, key).toString();  
            
            } else {
                System.out.println("Data retrieved from Aerospike cache");
                return record.toString();
                
            }
        }
}

In [31]:
Cache cache = new Cache();
cache.populateMongoData("id", "data");

In [33]:
cache.getData("id", "data");


Data retrieved from Aerospike cache


(gen:1),(exp:350622413),(bins:(value:data))